In [1]:
# Train CAM model with pretrained model in this case, resnet18 and resnet50
# Because data is not fit with resnet back data, imagenet

In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch.optim import lr_scheduler
from PIL import Image
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import sys
sys.path.append('../')
from random_state import set_seed
sys.path.remove('../')
import time

local_path = 'C:/Users/MZC01-HYUNHOPART/Desktop/PROJECT/PAPER/HAM10000'

# SET RANDOMSEED
set_seed(seed=42)

# Get Mean and Std about MAM10000
with open(f'{local_path}/HAM10000_MeanStd.json', 'r') as ms_json:
    MeanStd = json.load(ms_json)
    
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

print(f'IMPORTED....')

RANDOM STATE 42 IS STAIBLIZED....
IMPORTED....


In [2]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, label_dir, transform=None, augmented_images=None, augmented_labels=None):
        self.root_dir = root_dir
        self.metadata = pd.read_csv(label_dir)
        self.transform = transform

        # Create a mapping from string labels to integers
        self.label_mapping = {str(label): idx for idx, label in enumerate(self.metadata.iloc[:, 1].unique())}
        
        # Print unique labels for debugging
        print("Unique Labels in Dataset:", self.label_mapping)

        # Augmented data
        self.augmented_images = augmented_images if augmented_images is not None else []
        self.augmented_labels = augmented_labels if augmented_labels is not None else []

    def __len__(self):
        # Return the total number of samples, including both original and augmented
        return len(self.metadata) + len(self.augmented_images)

    def unique_class(self):
        return len(self.metadata.iloc[:, 1].unique())

    def __getitem__(self, idx):
        if idx < len(self.metadata):
            # Original data
            img_name = self.metadata.iloc[idx, 0]
            label_str = self.metadata.iloc[idx, 1]
            label = self.label_mapping[label_str]
            img_path = os.path.join(self.root_dir, img_name + '.jpg')
            image = Image.open(img_path).convert('RGB')
        else:
            # Augmented data
            idx -= len(self.metadata)
            image = self.augmented_images[idx]
            label = self.augmented_labels[idx]
            
        if self.transform:
            image = self.transform(image)

        return image, label


def load_finetune_dataset(data_dir:str, label_dir:str, batch_size:int=32, 
                          MeanStd:dict=MeanStd, num_workers:int=0, apply_augment:bool=True, apply_random:bool=True):
    '''
    Load target fintune dataset to fine tune on pre-trained model
    Argument
        - data_dir: The directory containing the dataset.
        - label_dir: The directory containing label data in csv file.
        - batch_size: The batch size for loading the dataset.    
        - meanstd : Computed the HAM10000 mean and standard deviation value  
    '''
    transform = transforms.Compose([
        transforms.Resize((224, 224)) # ImageNet Trained size
        , transforms.ToTensor()
        , transforms.Normalize(mean=MeanStd['Mean'], std=MeanStd['Std'])
    ])

    dataset = CustomDataset(data_dir, label_dir, transform=transform)
    unique_class = dataset.unique_class()

    # Split the dataset into training, validation and test sets
    train_size = int(0.8 * len(dataset))
    valid_size = int(0.1 * len(dataset))
    test_size  = len(dataset) - train_size - valid_size
        
    # Split train, valid, and test data
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

    if apply_augment:
        # Apply data augmentation only to the training dataset
        augmented_train_images, augmented_train_labels, randomness = apply_augmentation(train_dataset, randomness=apply_random)

        # Extend the training dataset with augmented images
        train_dataset.dataset.augmented_images = augmented_train_images
        train_dataset.dataset.augmented_labels = augmented_train_labels
        train_dataset.dataset.metadata = pd.concat([train_dataset.dataset.metadata,
                                                    pd.DataFrame({'Image': [f'image_{i}' for i in range(len(augmented_train_images))],
                                                                'Label': augmented_train_labels})],
                                                ignore_index=True)
    else:
        randomness = False

    # Print information
    print(f'Training dataset size: {len(train_dataset.dataset)}')
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True , num_workers=num_workers)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    return train_loader, valid_loader, test_loader, unique_class, apply_augment, randomness

def apply_augmentation(dataset, randomness:bool=True):
    augmentations = [
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
    ]

    augmented_images = []
    augmented_labels = []
    
    
    for image, label in tqdm(dataset):
        if randomness:
            # Apply a random augmentation to each image
            augmentation = random.choice(augmentations)
            augmented_images.append(transforms.ToPILImage()(augmentation(image)))
            augmented_labels.append(label)

        else :
            # Apply all augmentations to each image
            augmented_images.extend([transforms.ToPILImage()(augmentation(image)) for augmentation in augmentations])
            augmented_labels.extend([label] * len(augmentations))
    
    return augmented_images, augmented_labels, randomness

In [3]:
class EarlyStopping:
    def __init__(self, patience:int=5, threshold:float=0.1):
        self.patience  = patience
        self.threshold = threshold
        self.counter   = 0
        self.best_score= 0
        self.early_stop= False
    
    def __call__(self, val_accuracy):
        score = val_accuracy
        
        if score > self.best_score + self.threshold:
            self.counter   = 0
            self.best_score= score
        
        else:
            self.counter+=1
            if self.counter >= self.patience:
                self.early_stop = True
        
        return self.early_stop

In [4]:

%%time

train_loader, valid_loader, test_loader, unique_class, apply_augment, randomness = load_finetune_dataset(
    data_dir=f'{local_path}/images/', 
    label_dir=f'{local_path}/HAM10000_label.csv',
    batch_size=64, MeanStd=MeanStd, num_workers=0, apply_augment=False, apply_random=True,
    )

print(f'DATA LOADER IS COMPLETED....\n')

Unique Labels in Dataset: {'bkl': 0, 'nv': 1, 'df': 2, 'mel': 3, 'vasc': 4, 'bcc': 5, 'akiec': 6}
Training dataset size: 10015
DATA LOADER IS COMPLETED....

CPU times: total: 0 ns
Wall time: 14 ms


In [5]:
model_architecture = str(input("Please select model architecture betweeb resnet18, resnet50 and resnet101."))

if model_architecture not in ['resnet18', 'resnet50', 'resnet101']:
    raise ValueError(f"Model architecture '{model_architecture}' should be 'resnet18', 'resnet50' and 'resnet101'.")
else:
    print(f'Model set as {model_architecture}......')

Model set as resnet50......


In [6]:

# Trainig Loop
num_epochs = int(input("How many times to fine tune the model? e.g., 10, 20, 30"))

print(f'MODEL WILL FINE TUNE BY {num_epochs} TIMES.....')

MODEL WILL FINE TUNE BY 100 TIMES.....


In [7]:
# Load pre-trained Resnet18 and Resnet50 for training

model_class = getattr(models, model_architecture)
model = model_class(weights=True)
model.fc = nn.Linear(model.fc.in_features, unique_class)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) # can change sgd for computation power
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Use GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f'PYTORCH DEVICE : {device}')


PYTORCH DEVICE : cuda:0



In [8]:
%%time

#early_stopping = EarlyStopping(patience=10, threshold=0.1)

# Training
for epoch in tqdm(range(num_epochs), desc="Training Progress", unit="epoch"):
    model.train()
    total_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Average training loss for the epoch
    average_loss = total_loss / len(train_loader)
    
    # update the learning rate after every epoch
    scheduler.step()
        
    # Validation check by every step and last one
    step:int = 1
    if (epoch % step == step-1) or (epoch == num_epochs-1):
        model.eval()
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        # Calculate validation accuracy
        val_accuracy = accuracy_score(all_labels, all_preds) * 100
        tqdm.write(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {average_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%...')

        # Check the early stopping
        # if early_stopping(val_accuracy):
        #     print('Early Stopped....')
        #     break

Training Progress:   1%|          | 1/100 [12:25<20:30:09, 745.56s/epoch]

Epoch 1/100, Training Loss: 0.8635, Validation Accuracy: 70.03%...


Training Progress:   2%|▏         | 2/100 [26:16<21:39:31, 795.63s/epoch]

Epoch 2/100, Training Loss: 0.7430, Validation Accuracy: 41.16%...


Training Progress:   3%|▎         | 3/100 [39:56<21:44:21, 806.82s/epoch]

...

Epoch 98/100, Training Loss: 0.0714, Validation Accuracy: 82.02%...


Training Progress:  99%|█████████▉| 99/100 [21:54:54<14:08, 848.83s/epoch]

Epoch 99/100, Training Loss: 0.0723, Validation Accuracy: 82.02%...


Training Progress: 100%|██████████| 100/100 [22:09:05<00:00, 797.46s/epoch]

Epoch 100/100, Training Loss: 0.0739, Validation Accuracy: 82.12%...
CPU times: total: 8h 36min 18s
Wall time: 22h 9min 5s


In [9]:
# Test
model.eval()
test_preds, test_labels = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# Calculate test accuracy
test_accuracy = accuracy_score(test_labels, test_preds)
print(f'Test Accuracy : {test_accuracy:.2f}%...')

Test Accuracy : 0.80%...


In [10]:
# Save the fine-tuned model

if apply_augment:
    pth_name = f'./fine_tune_model/{model_architecture}_iter_{num_epochs}_aug_{apply_augment}_rand_{randomness}_test_{test_accuracy:.2f}%.pth'
else:
    pth_name = f'./fine_tune_model/{model_architecture}_iter_{num_epochs}_aug_{apply_augment}_test_{test_accuracy:.2f}%.pth'    

torch.save(model.state_dict(), pth_name)
    
print(f'Pretrained model saved.....')

Pretrained model saved.....
